In [1]:
from bs4 import BeautifulSoup as BS
from urllib.request import urlopen
import re

def readpage(url):
    html_mainfile = urlopen(url)
    text_mainfile = html_mainfile.read()
    soup = BS(text_mainfile,'html.parser')
    return soup

mainpage = 'https://www.century21.com/property-search-map-California-SCA'
soup_mainfile = readpage(mainpage)
soup_list=soup_mainfile.findAll('a') # To narrow the search because all <a> contain href.


In [2]:
county_list = ['alameda','contra-costa','marin','san-francisco','napa',
               'san-mateo','santa-clara','solano','sonoma']

website_url = 'https://www.century21.com'
regex = '''href="(.*?)" id'''
county_url_lst = []

for i in soup_list:
    i = str(i)
    for j in county_list:
        if j in i:
            county_url_lst.append(re.findall(regex,i)[0])
    #We do not need the last county since san francisco is already in item 3
    county_url_lst = county_url_lst[:9]

whole_url = []
for i in county_url_lst:
    i = f"{website_url}{i}"
    whole_url.append(i)
    print(i) #These are all the url of the counties we need to scrape

https://www.century21.com/real-estate/san-mateo-county-ca/LNCASANMATEO/
https://www.century21.com/real-estate/santa-clara-county-ca/LNCASANTACLARA/
https://www.century21.com/real-estate/san-francisco-county-ca/LNCASANFRANCISCO/
https://www.century21.com/real-estate/alameda-county-ca/LNCAALAMEDA/
https://www.century21.com/real-estate/napa-county-ca/LNCANAPA/
https://www.century21.com/real-estate/contra-costa-county-ca/LNCACONTRACOSTA/
https://www.century21.com/real-estate/solano-county-ca/LNCASOLANO/
https://www.century21.com/real-estate/marin-county-ca/LNCAMARIN/
https://www.century21.com/real-estate/sonoma-county-ca/LNCASONOMA/


In [3]:
def findAddress(soup):
    prop_address = soup.findAll('div',attrs={'class':'property-address-info'})
    prop_address = str(prop_address)
    title = re.findall('title="(.*?)"',prop_address)
    return title
    
def setBlock(address,soup):
    soup = str(soup)
    regex = '<div class="property-address" title="{}">'.format(address)
    index = re.search(regex,soup).start()
    soup = soup[index:]
    try:
        index2 = re.search('<div class="property-address-info">',soup).start()
    except:
        index2 = len(soup)
    soup = soup[:index2]
    return soup


def getBlock(Url):
    Soup = readpage(Url)
    block = []
    for i in findAddress(Soup):
        block.append(setBlock(i,Soup))
    block = block[:len(block)-1] # remove last element because no data in it
    return block
    
getBlock('https://www.century21.com/real-estate/san-mateo-county-ca/LNCASANMATEO/')[18]

'<div class="property-address" title="1547 Tarrytown St">\n               1547 Tarrytown St\n            </div>\n<div class="property-city">\n            San Mateo CA 94402\n         </div>\n</div>\n<div class="property-card-attribution">\n               Courtesy Of Coldwell Banker Burlingame\n            \n         </div>\n</div>\n<div class="property-actions">\n<button class="action-contact" data-usagetrack=\'{"data": ["Hybrid Mapping", "Property Card", "Contact"]}\' title="Contact" type="button"></button>\n<button class="action-favorite" title="Set as Favorite" type="button"></button>\n</div>\n</div>\n<div class="infinite-item property-card clearfix property-card-CBR51801695" data-brand-cd="CBR" data-id="CBR51801695" data-latitude="37.492459" data-link="/property/14-calypso-lane-san-carlos-ca-94070-CBR51801695" data-listing-id="" data-longitude="-122.294095" data-mls="ML81834917" data-source-id="ED6DCCD0-B428-4DF5-98D3-C5A89539C845" data-zip="94070">\n<div class="property-card-clip"

In [4]:
import pandas as pd

def Extractor(block,url):
    lib = {}
    PropAdd = re.findall('title="(.*?)"',block)
    City = re.findall('property-city">\n\s*(.*?)\sCA\s\d{5}\n',block)
    ZipCode = re.findall('CA (\d{5})',block)
    Agent = re.findall('Courtesy Of (.*?)\n',block)
    ForSale = re.findall('pdp-listing-type sale">(.*?)</div>',block)
    Price = re.findall('(\$[0-9\,]+)\n',block)
    Beds = re.findall('<strong>(\d.*)</strong> beds',block)
    Baths = re.findall('<strong>(\d.*)</strong> bath',block)
    HalfBaths = re.findall('<strong>(\d.*)</strong> half bath',block)
    SqFt = re.findall('<strong>(.*?)</strong> sq. ft',block)
    lib['Property Address']=PropAdd
    lib['City']=City
    lib['Zip Code']=ZipCode
    lib['Agent']=Agent
    lib['For Sale']=ForSale
    lib['Price']=Price
    lib['Beds']=Beds
    lib['Baths']=Baths
    lib['Half Baths']=HalfBaths
    lib['Sq Ft']=SqFt
    lib['County']=re.findall('real-estate/(.*?)-ca/',url)
    return lib

lst = []
for url in whole_url:
    blocks = getBlock(url)
    for blk in blocks:
        lst.append(Extractor(blk,url))
    
df = pd.DataFrame(lst)


In [5]:
import numpy as np
import os

os.chdir(r'C:\Users\limso\Downloads')
df2 = pd.DataFrame()
for i in ['Property Address', 'City', 'Zip Code', 'Agent', 'For Sale', 'Price','Beds', 'Baths', 'Half Baths', 'Sq Ft','County']:
    df2[i] = df[i].apply(lambda y: y[0] if len(y)!=0 else np.nan)


In [6]:
#Data Cleaning

df2['Agent']=df2['Agent'].fillna('None')
df2['Half Baths']=df2['Half Baths'].fillna(0)

'''
Below is the average psf I calculated based on every county. I will use these psf to calculate 
the missing sq ft.

alameda-county	 649.37 
contra-costa-county	 559.49 
marin-county	 690.69 
napa-county	 686.89 
san-francisco-county	 920.39 
san-mateo-county	 926.31 
santa-clara-county	 685.28 
solano-county	 353.76 
sonoma-county	 434.60 

'''

def removesym(string):
    string=string.replace('$','')
    string=string.replace(',','')
    return int(string)

df2['Price']=df2['Price'].apply(lambda x: removesym(x))

def calc_sf(price,county,sf):
    if sf is np.nan:   
        if county == 'alameda-county':
            new_sf = price/649.37
        elif county == 'contra-costa-county':
            new_sf = price/559.49
        elif county == 'marin-county':
            new_sf = price/690.69
        elif county == 'napa-county':
            new_sf = price/686.89
        elif county == 'san-francisco-county':
            new_sf = price/920.39
        elif county == 'san-mateo-county':
            new_sf = price/926.31
        elif county == 'santa-clara-county':
            new_sf = price/685.28
        elif county == 'solano-county':
            new_sf = price/353.76
        elif county == 'sonoma-county':
            new_sf = price/434.60
        return int(new_sf)
    else:
        return sf

df2['Sq Ft'] = df2.apply(lambda x: calc_sf(x['Price'],x['County'],x['Sq Ft']),axis=1)


In [7]:
#Remove rows that have both NA's for bed and bath
df2[df2['Beds'].isna()][df2['Baths'].isna()]
df3=df2.dropna(subset=['Beds', 'Baths'])

C:\Users\limso\anaconda3\envs\PythonCPU\lib\site-packages\ipykernel_launcher.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  


In [8]:
#Row 43 is a street name instead of a city. I want to keep this data, so I will
#change the property address and city of this row
print(df3.iloc[43])
df3.iloc[[43],[0]] = "530 Kirkham Street"
df3.iloc[[43],[1]] = "San Francisco"
print('\n')
print(df3.iloc[43])

Property Address                     530
City                      Kirkham Street
Zip Code                           94122
Agent                               None
For Sale                        FOR SALE
Price                            1199000
Beds                                   2
Baths                                  2
Half Baths                             0
Sq Ft                              1,074
County              san-francisco-county
Name: 50, dtype: object


Property Address      530 Kirkham Street
City                       San Francisco
Zip Code                           94122
Agent                               None
For Sale                        FOR SALE
Price                            1199000
Beds                                   2
Baths                                  2
Half Baths                             0
Sq Ft                              1,074
County              san-francisco-county
Name: 50, dtype: object


C:\Users\limso\anaconda3\envs\PythonCPU\lib\site-packages\pandas\core\indexing.py:1765: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)


In [10]:
#Remove Duplicates
df_final=df3.drop_duplicates(subset='Property Address', keep='first', inplace=False)
df_final.to_csv('output.csv')
df_final

,Property Address,City,Zip Code,Agent,For Sale,Price,Beds,Baths,Half Baths,Sq Ft,County
0,110/112 San Antonio AVE,San Bruno,94066,Better Homes and Gardens Real Estate J.F. Finn...,FOR SALE,1299999,3,3,0,"1,490",san-mateo-county
1,1084 Park Pacifica Avenue,Pacifica,94044,Keller Williams East County,FOR SALE,1228000,3,2,0,"1,080",san-mateo-county
2,2107 Shoreview Ave,San Mateo,94401,REALTY EXPERTS,FOR SALE,289000,2,2,0,992,san-mateo-county
3,3015 E. Bayshore Rd. #439,Redwood City,94063,Alliance Manufactured Homes,FOR SALE,949950,2,2,0,"1,213",san-mateo-county
4,800 Sea Spray Lane Unit 212,Foster City,94404,None,FOR SALE,3198000,5,5,1,"3,918",san-mateo-county
...,...,...,...,...,...,...,...,...,...,...,...
166,1416 Casa Grande Road,Petaluma,94954,None,FOR SALE,820000,3,2,0,"2,007",sonoma-county
167,2869 Spyglass Court,Santa Rosa,95405,None,FOR SALE,329000,2,1,0,828,sonoma-county
168,3851 Sebastopol Road #104,Santa Rosa,95407,Coldwell Banker Santa Rosa - Bicentennial,FOR SALE,202000,3,2,0,"1,296",sonoma-county
169,300 Candy Ln #59,Santa Rosa,95403,Coldwell Banker Santa Rosa - Bicentennial,FOR SALE,779000,2,2,0,"1,502",sonoma-county
